<a href="https://colab.research.google.com/github/ming-zhao/Business-Analytics/blob/master/files/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -q https://raw.githubusercontent.com/ming-zhao/Business-Analytics/master/files/loaddatfuncs.py > /dev/null
!wget -q https://raw.githubusercontent.com/ming-zhao/Business-Analytics/master/files/loadmlfuncs.py > /dev/null
!wget -q https://raw.githubusercontent.com/ming-zhao/Business-Analytics/master/files/loadoptfuncs.py > /dev/null
!wget -q https://raw.githubusercontent.com/ming-zhao/Business-Analytics/master/files/loadregfuncs.py > /dev/null
!wget -q https://raw.githubusercontent.com/ming-zhao/Business-Analytics/master/files/loadtsfuncs.py > /dev/null
!pip install imgaug==0.2.5 > /dev/null
!pip install scipy==1.2 > /dev/null
!pip install pmdarima > /dev/null
%run loaddatfuncs.py
%run loadmlfuncs.py
%run loadoptfuncs.py
%run loadregfuncs.py
%run loadtsfuncs.py

# Forecasting

In [ ]:
interact(training_data,
         show=widgets.Checkbox(value=False, description='Original', disabled=False));

In [ ]:
interact(poly_fit,
         show=widgets.Checkbox(value=True, description='sin$(2\pi x)$', disabled=False));

In [ ]:
interact(poly_fit_holdout,
         show=widgets.Checkbox(value=True, description='sin$(2\pi x)$', disabled=False),
         train=widgets.Checkbox(value=True, description='Training Data', disabled=False),
         test=widgets.Checkbox(value=True, description='Testing Data', disabled=False));

In [ ]:
boxplot(df_house, ['month','sales'], title='House Sales')

# Machine Learning

## Naive Bayes

In [ ]:
from sklearn.datasets import fetch_20newsgroups

data = fetch_20newsgroups()
display(data.target_names)

# choose a subset categories to learn
categories = ['talk.religion.misc',
              'soc.religion.christian',
              'sci.space',
              'comp.graphics']

train = fetch_20newsgroups(subset='train', categories=categories)
test = fetch_20newsgroups(subset='test', categories=categories)

In [ ]:
print(train.data[5])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix

# Fit the model and show the classification matrix
#convert the content of each string into a vector of numbers
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(train.data, train.target)
labels = model.predict(test.data)
mat = confusion_matrix(test.target, labels)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=train.target_names, yticklabels=train.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label')
plt.ylim(0, mat.shape[0])
plt.show()

def predict_category(s, train=train, model=model):
    pred = model.predict([s])
    return train.target_names[pred[0]]

In [ ]:
predict_category('discussing islam vs atheism')

In [ ]:
predict_category('determining the screen resolution')

In [ ]:
predict_category('sending a payload to the ISS')

## Neural Networks

In [ ]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print('training images:{}, test images:{}'.format(train_images.shape, test_images.shape))

In [ ]:
def showimg(data, idx):
    span = 5
    if data=='train':
        if idx+span<train_images.shape[0]:
            images = train_images
            labels = train_labels
        else:
            print('Index is out of range.')
    if data=='test':
        if idx+span<test_images.shape[0]:
            images = test_images
            labels = test_labels
        else:
            print('Index is out of range.')
    plt.figure(figsize=(20,4))
    for i in range(span):
        plt.subplot(1, 5, i + 1)
        digit = images[idx+i]
        plt.imshow(digit, cmap=plt.cm.binary)
        plt.title('Index:{}, Label:{}'.format(idx+i, labels[idx+i]), fontsize = 15)
    plt.show()

interact(showimg,
    data = widgets.RadioButtons(options=['train', 'test'],
                                value='train', description='Data:', disabled=False),
    idx = widgets.IntText(value=7, description='Index:', disabled=False));

In [ ]:
from keras import models
from keras import layers
from keras.utils import to_categorical

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

train_images_reshape = train_images.reshape((60000, 28 * 28))
train_images_reshape = train_images_reshape.astype('float32') / 255

test_images_reshape = test_images.reshape((10000, 28 * 28))
test_images_reshape = test_images_reshape.astype('float32') / 255

train_labels_cat = to_categorical(train_labels)
test_labels_cat = to_categorical(test_labels)

network.fit(train_images_reshape, train_labels_cat, epochs=5, batch_size=128)

test_loss, test_acc = network.evaluate(test_images_reshape, test_labels_cat)
print('test accuracy:', test_acc)

In [ ]:
predicted = network.predict_classes(test_images_reshape)
result = abs(predicted - test_labels)
misclassified = np.where(result>0)[0]
print('# of misclassified images:',misclassified.shape[0])

plt.figure(figsize=(20,4))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    idx = misclassified[i]
    digit = test_images[idx]
    plt.imshow(digit, cmap=plt.cm.binary)
    plt.title('Predicted:{}, Label:{}'.format(predicted[idx], test_labels[idx]), fontsize = 15)
plt.show()